In [15]:
import pickle
from src.clf.constants import LR_MT_CLASSIFIED, RF_MT_CLASSIFIED, DT_MT_CLASSIFIED, SVM_MT_CLASSIFIED, XGB_MT_CLASSIFIED
from src.clf.constants import LR_MT_MASKED, RF_MT_MASKED, DT_MT_MASKED, SVM_MT_MASKED, XGB_MT_MASKED
from sklearn import set_config

In [2]:
# get access to a folder outside of the src folder
import os
os.chdir('..')

In [21]:
# import model and display pipeline
set_config(display='diagram')
model = pickle.load(open(SVM_MT_MASKED, 'rb'))
model

Pipeline(steps=[('vect', CountVectorizer()), ('smote', SMOTE(random_state=42)),
                ('clf',
                 SVC(C=1, gamma=0.01, probability=True, random_state=42))])

In [10]:
# show the model's parameters
model.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer()),
  ('smote', SMOTE(random_state=42)),
  ('clf',
   DecisionTreeClassifier(max_depth=17, max_features=0.3, random_state=42))],
 'verbose': False,
 'vect': CountVectorizer(),
 'smote': SMOTE(random_state=42),
 'clf': DecisionTreeClassifier(max_depth=17, max_features=0.3, random_state=42),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': None,
 'vect__vocabulary': None,
 'smote__k_neighbors': 5,
 'smote__n_jobs': None,
 'smote__random_state': 42,
 'smote__sampling_strategy': 'auto',
 'clf__ccp_alpha': 0.0,
 'clf__class_weight': None,
 'clf__cri

In [18]:
# show criterion of the model
model['clf'].criterion

'entropy'

In [6]:
# import mtsamples_finaloutput.csv as pandas dataframe
import pandas as pd
df = pd.read_csv('data/processed/nlp/mtsamples/mtsamples_finaloutput.csv')
# print full column width
pd.set_option('display.max_colwidth', None)
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,nr_candidates,top_n,keywords_outcome_weights_MLM,transcription_f_MLM,keywords_outcome_weights_TC,transcription_f_TC
0,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,mmode atrial enlargement atrial diameter normal size right ventricle normal lv systolic function ventricular ejection fraction normal lv diastolic function pericardial effusion normal morphology aortic valve mitral valve tricuspid valve pulmonary valve pa systolic pressure mmhgdoppler mild mitral tricuspid regurgitation trace aortic pulmonary regurgitation,"cardiovascular / pulmonary, 2-d m-mode, doppler, aortic valve, atrial enlargement, diastolic function, ejection fraction, mitral, mitral valve, pericardial effusion, pulmonary valve, regurgitation, systolic function, tricuspid, tricuspid valve, normal lv","['cardiovascular / pulmonary', ' 2-d m-mode', ' doppler', ' aortic valve', ' atrial enlargement', ' diastolic function', ' ejection fraction', ' mitral', ' mitral valve', ' pericardial effusion', ' pulmonary valve', ' regurgitation', ' systolic function', ' tricuspid', ' tricuspid valve', ' normal lv ']","dict_values([[221, 233], [11, 29], [163, 181], [135, 152], [234, 240], [234, 246], [182, 202], [263, 278], [334, 347], [99, 116], [247, 256], [247, 262], [89, 99]])",5,2,"[('tricuspid regurgitation', 0.882), ('pericardial effusion', 0.8454)]","['tricuspid regurgitation', 'pericardial effusion']","[('lv diastolic', 0.8428), ('effusion normal', 0.8311)]","['lv diastolic', 'effusion normal']"
1,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,ventricular cavity size wall thickness normal wall motion ventricular systolic function hyperdynamic estimate ejection fraction nearcavity obliteration increase ventricular outflow tract gradient mid cavity level consistent hyperdynamic ventricular systolic function abnormal ventricular relaxation pattern well elevate atrial pressure doppler atrium mildly dilate right atrium right ventricle normal aorti,"cardiovascular / pulmonary, 2-d, doppler, echocardiogram, annular, aortic root, aortic valve, atrial, atrium, calcification, cavity, ejection fraction, mitral, obliteration, outflow, regurgitation, relaxation pattern, stenosis, systolic function, tricuspid, valve, ventricular, ventricular cavity, wall motion, pulmonary artery","['cardiovascular / pulmonary', ' 2-d', ' doppler', ' echocardiogram', ' annular', ' aortic root', ' aortic valve', ' atrial', ' atrium', ' calcification', ' cavity', ' ejection fraction', ' mitral', ' obliteration', ' outflow', ' regurgitation', ' relaxation pattern', ' stenosis', ' systolic function', ' tricuspid', ' valve', ' ventricular', ' ventricular cavity', ' wall motion', ' pulmonary artery']","dict_values([[409, 416], [680, 687], [506, 517], [532, 544], [389, 395], [435, 441], [688, 701], [17, 23], [135, 152], [673, 679], [164, 176], [220, 227], [781, 794], [347, 365], [578, 586], [88, 105], [795, 804], [539, 544], [5, 16], [5, 23], [58, 69], [864, 880]])",5,2,"[('hyperdynamic ventricular', 0.8962), ('ventricular outflow', 0.8705)]","['hyperdynamic ventricular', 'ventricular outflow']","[('consistent hyperdynamic', 0.883), ('wall motion', 0.8099)]","['consistent hyperdynamic', 'wall motion']"
2,7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,echocardiogrammultiple view heart great vessels normal intracardiac great vessel relationships cardiac function normal chamber enlargement hypertrophy pericardial effusion vegetations doppler interrogation color flow systemic venous return right atrium normal tricuspid inflow pulmonary outflow normal valve pulmonary venous return atrium interatrial septum intact mitral inflow ascend aorta flow normal aortic valve trileaflet coronary arteries normal or,"cardiovascular / pulmonary, 2-d echocardiogram, cardiac function, doppler, echocardiogram, multiple views, aortic valve, coronary arteri

In [11]:
# show rows where  "salpingoo" is in transcription_f_MLM
df[df['transcription_f_MLM'].str.contains('salpingoo')]


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,nr_candidates,top_n,keywords_outcome_weights_MLM,transcription_f_MLM,keywords_outcome_weights_TC,transcription_f_TC
266,302,Total abdominal hysterectomy (TAH) with a right salpingo-oophorectomy.,Surgery,TAH & Salpingo-oophorectomy - 1,preoperative persistent abnormal uterine bleed endometrial ablationpostoperative persistent abnormal uterine bleed endometrial ablationprocedure abdominal hysterectomy tah right salpingooophorectomycomplications noneestimated blood loss ccfluids crystalloidsurine urine end procedurefindings anesthesia obese female enlarge fibroid uterus freely movable pelvis operative demonstrate normal bilaterally right ovary contain right ovarian cyst lea,"surgery, tah, salpingo-oophorectomy, total abdominal hysterectomy, abnormal uterine bleeding, endometrial ablatio, ochsner clamps, round ligaments, clamps, ligaments, cuff, salpingo, oophorectomy, hysterectomy, uterine, ablation, tubes, abdominal, anesthesia,","['surgery', ' tah', ' salpingo-oophorectomy', ' total abdominal hysterectomy', ' abnormal uterine bleeding', ' endometrial ablatio', ' ochsner clamps', ' round ligaments', ' clamps', ' ligaments', ' cuff', ' salpingo', ' oophorectomy', ' hysterectomy', ' uterine', ' ablation', ' tubes', ' abdominal', ' anesthesia', '']","dict_values([[201, 204], [172, 200], [56, 75], [1858, 1873], [1864, 1873], [3077, 3081], [211, 219], [188, 200], [42, 49], [68, 76], [178, 187], [342, 352], [0, -1]])",5,2,"[('endometrial ablationprocedure', 0.9277), ('salpingooophorectomycomplications noneestimated', 0.9254)]","['endometrial ablationprocedure', 'salpingooophorectomycomplications noneestimated']","[('ovary contain', 0.9033), ('abnormal uterine', 0.8873)]","['ovary contain', 'abnormal uterine']"
267,303,Total abdominal hysterectomy (TAH) with bilateral salpingooophorectomy and uterosacral ligament vault suspension. Cervical intraepithelial neoplasia grade-III postconization. Recurrent dysplasia. Uterine procidentia grade II-III. Mild vaginal vault prolapse.,Surgery,TAH & Salpingooophorectomy,preoperative cervical intraepithelial neoplasia gradeiii post conization poor margins recurrent dysplasia unable office uterine procidentia grade iiiiipostoperative cervical intraepithelial neoplasia gradeiii postconization poor margins recurrent dysplasia uterine procidentia grade iiiii mild vaginal vault prolapseprocedures abdominal hysterectomy tah bilateral salpingooophorectomy uterosacral ligament vault suspensionanesthesia spinal astramorph postope,"surgery, cervical intraepithelial neoplasia, vaginal vault prolapse, uterosacral ligament vault suspension, total abdominal hysterectomy, bilateral salpingooophorectomy, abdominal hysterectomy, uterosacral ligament, recurrent dysplasia, uterine procidentia, suture ligated, abdominal, intraepithelial, tah, salpingooophorectomy, hysterectomy, ligament, hemostats, vaginal,","['surgery', ' cervical intraepithelial neoplasia', ' vaginal vault prolapse', ' uterosacral ligament vault suspension', ' total abdominal hysterectomy', ' bilateral salpingooophorectomy', ' abdominal hysterectomy', ' uterosacral ligament', ' recurrent dysplasia', ' uterine procidentia', ' suture ligated', ' abdominal', ' intraepithelial', ' tah', ' salpingooophorectomy', ' hysterectomy', ' ligament', ' hemostats', ' vaginal', ' ']","dict_values([[21, 55], [325, 347], [430, 467], [366, 394], [399, 429], [372, 394], [430, 450], [101, 120], [142, 161], [372, 381], [30, 45], [395, 398], [409, 429], [382, 394], [442, 450], [2133, 2142], [325, 332], [12, 12]])",5,2,"[('salpingooophorectomy uterosacral', 0.9017), ('iiiiipostoperative cervical', 0.8947)]","['salpingooophorectomy uterosacral', 'iiiiipostoperative cervical']","[('uterine', 0.875), ('vaginal vault', 0.8586)]","['uterine', 'vaginal vault']"
268,304,"Total abdominal hysterectomy (TAH), left salpingo-oophorectomy, lysis of interloop bowel adhesions. Chronic pelvi

In [12]:
# load dt_mt_masked.pkl from models as model and display lime explanation
import lime
import lime.lime_tabular
import pickle

model = pickle.load(open('models/clf/dt_mt_masked.pkl', 'rb'))


ImportError: cannot import name 'DT_MODEL_MASKED' from 'src.clf.constants' (/Users/hannahpetry/Desktop/Work Project/NLP_Masterthesis/src/clf/constants.py)